In [ ]:
using Revise

In [ ]:
using MeshCatMechanisms
using MeshCat
using RigidBodyDynamics
using RigidBodySim
using Flux
using JLD2
using Plots
using LCPSim
using LearningMPC
using LearningMPC.Models
using DataFrames
using Blink

In [ ]:
interval_net_param, lqrsol = jldopen("cartpole-interval-wall-params.jld2") do file
    file["net"], file["lqrsol"]
end

interval_net = jldopen("../2018-08-31-more-pendulum/cartpole-interval.jld2") do file
    file["net"]
end

In [ ]:
@load "library.jld2" training

In [ ]:
length(training)

In [ ]:
typeof(training)

In [ ]:
xx = linspace(-8, 8, 51)
yy = linspace(0.5, 2.0, 51)
getstate(x, y) = [0, 0, y, x, 0]
plt = heatmap(xx, yy, (x, y) -> (getstate(x, y) - lqrsol.x0)' * lqrsol.S * (getstate(x, y) - lqrsol.x0), 
    color=:coolwarm,
    clim=(0, 15000))
title!(plt, "LQR Cost-to-Go")
# xlabel!(plt, "Initial x velocity")
# ylabel!(plt, "Initial rotational velocity")
Plots.savefig(plt, "lqr_cost_to_go.pdf")
plt

In [ ]:
plt = heatmap(xx, yy, (x, y) -> Flux.Tracker.data(interval_net(getstate(x, y)[[1,2,4,5]]))[], color=:coolwarm,
    clim=(0, 4000))
title!(plt, "Learned Cost-to-Go")
# xlabel!(plt, "Initial x velocity")
# ylabel!(plt, "Initial rotational velocity")
Plots.savefig(plt, "learned_cost_to_go.pdf")
plt

In [ ]:
gr()
plt = heatmap(xx, yy, 
    (x, y) -> Flux.Tracker.data(interval_net_param(getstate(x, y)))[], color=:coolwarm,
#     clim=(0, 6000),
)
title!(plt, "Learned Cost-to-Go")
xlabel!(plt, "Initial cart velocity")
ylabel!(plt, "Distance to each wall")
Plots.savefig(plt, "learned_cost_to_go_param.pdf")
plt

In [ ]:
gr()
plt = heatmap(xx, yy, 
    (x, y) -> Flux.Tracker.data(interval_net_param(getstate(x, y)))[], color=:coolwarm,
    clim=(0, 6000))
xl = xlims(plt)
yl = ylims(plt)
xa = linspace(0, 5)
ya = xa .* 0.5
plot!(plt, xa, ya, color=:yellow, linewidth=3, label="")
plot!(plt, .-xa, ya, color=:yellow, linewidth=3, label="")
xlims!(plt, xl)
ylims!(plt, yl)
title!(plt, "Learned Cost-to-Go")
# xlabel!(plt, "Initial x velocity")
# ylabel!(plt, "Initial rotational velocity")
Plots.savefig(plt, "learned_cost_to_go_param_annotated.pdf")
plt

In [ ]:
pyplot()
f = (x, y) -> Flux.Tracker.data(interval_net_param(getstate(x, y)))[]
plt = Plots.contourf(xx, yy, f,
    color=:coolwarm,
    legend=:bottomright,
#     clim=(0, 6000),
#     nlevels=20,
#     levels=collect(0:500:6000),
)
Plots.contour!(plt, xx, yy, f, color=cgrad([:black, :black]))
xl = xlims(plt)
yl = ylims(plt)
xa = linspace(0, 5)
ya = xa .* 0.5
plot!(plt, xa, ya, color=:yellow, linewidth=3, label="y = ±0.5 x")
plot!(plt, .-xa, ya, color=:yellow, linewidth=3, label="")
xlims!(plt, xl)
ylims!(plt, yl)
xlabel!(plt, "Initial cart velocity")
ylabel!(plt, "Distance to each wall")
Plots.savefig(plt, "learned_cost_contour_annotated.pdf")
plt

In [ ]:
using Colors

In [ ]:
colorant"#fbb4ae"

In [ ]:
pyplot()
plt = wireframe(xx, yy, 
    (x, y) -> Flux.Tracker.data(interval_net_param(getstate(x, y)))[], 
#     color=:coolwarm,
#     color=colorant"#b3cde3",
#     clim=(0, 4000),
#     alpha=0.9,
    size=(700, 600),
    camera=(5, 30),
    label="Learned cost (parametric environment)",
    legend=:topleft,
#     background_color=RGBA(1, 1, 1, 0),
)
xl = xlims(plt)
yl = ylims(plt)
zl = zlims(plt)

xa = linspace(1, 4)
ya = xa .* 0.5
za = ((x, y) -> Flux.Tracker.data(interval_net_param(getstate(x, y)))[]).(xa, ya) .+ 1
plot!(plt, xa, ya, za, color=:yellow, linewidth=3, label="y = ±0.5 x")
plot!(plt, .-xa, ya, za, color=:yellow, linewidth=3, label="")

f = (x) -> Flux.Tracker.data(interval_net([0, 0, x, 0]))[]
plot!(plt, xx, fill(1.5, length(xx)), f.(xx), linewidth=2, color=:black, label="Learned cost (fixed environment)")

f = (x) -> ([0, 0, 1.5, x, 0] - lqrsol.x0)' * lqrsol.S * ([0, 0, 1.5, x, 0] - lqrsol.x0)
plot!(plt, xx, fill(1.5, length(xx)), f.(xx), linewidth=2, color=colorant"#e41a1c", label="LQR cost")


xlims!(plt, xl)
ylims!(plt, yl)
zlims!(plt, zl)
xlabel!(plt, "Initial cart velocity")
ylabel!(plt, "Distance to each wall")
Plots.savefig(plt, "learned_param_cost_3d.pdf")
plt

In [ ]:
plt = surface(xx, yy, 
    (x, y) -> Flux.Tracker.data(interval_net_param(getstate(x, y)))[], color=:coolwarm,
    clim=(0, 4000),
#     alpha=0.9
)
xl = xlims(plt)
yl = ylims(plt)
xa = linspace(0, 5)
ya = xa .* 0.5
za = ((x, y) -> Flux.Tracker.data(interval_net_param(getstate(x, y)))[]).(xa, ya)
plot!(plt, xa, ya, za, color=:yellow, linewidth=5, label="")
plot!(plt, .-xa, ya, za, color=:yellow, linewidth=5, label="")
xlims!(plt, xl)
ylims!(plt, yl)
plt

In [ ]:
gr()
plt = heatmap(xx, yy, 
    (x, y) -> Flux.Tracker.data(interval_net_param([0, 0, y, 0, x]))[], color=:coolwarm,
    clim=(0, 4000))
title!(plt, "Learned Cost-to-Go")